# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [11]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
dataset = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1 UBA Maestría DS\1_Especialización\1_Segundo Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_01.csv", index_col=0)

In [14]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto('C:/Users/Admin/Documents/1_Notebook/1_Estudio/1 UBA Maestría DS/1_Especialización/1_Segundo Semestre/DMEyF/Python/competencia_01.csv')



,Success


In [15]:
%%sql
select
    Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
from competencia_01 limit 10

,Master_Fvencimiento,Visa_Fvencimiento,tc_fvencimiento_mayor,tc_fvencimiento_menor
0,-1672.0,-1641.0,-1641.0,-1672.0
1,-2190.0,-2190.0,-2190.0,-2190.0
2,NaN,-1641.0,-1641.0,-1641.0
3,-729.0,-1337.0,-729.0,-1337.0
4,-514.0,-545.0,-514.0,-545.0
5,-1094.0,-1460.0,-1094.0,-1460.0
6,-545.0,-423.0,-423.0,-545.0
7,-272.0,-879.0,-272.0,-879.0
8,-1184.0,-1549.0,-1184.0,-1549.0
9,-637.0,-818.0,-637.0,-818.0


Lo siguiente es querer operar dos variables, como por ejemplo sumarla. Esto es sencillo


In [16]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,34658.40,162893.96,197552.36
1,37015.62,-85329.65,-48314.03
2,NaN,30199.73,NaN
3,0.00,6599.16,6599.16
4,0.00,35745.17,35745.17
5,0.00,4251.32,4251.32
6,1718.50,12648.93,14367.43
7,2132.94,6689.14,8822.08
8,0.00,5310.61,5310.61
9,0.00,24290.45,24290.45


Pero un DS de a de veras mirará los datos y se encontrará con un campo que es null cuando se lo suma a otro dará null.

In [17]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,NaN,30199.73,NaN
1,NaN,0.00,NaN
2,NaN,41073.06,NaN
3,NaN,5623.90,NaN
4,NaN,14673.34,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,13518.31,NaN
8,NaN,102359.25,NaN
9,NaN,NaN,NaN


Esto no siempre es deseable y puede ser fácilmente evitable

In [18]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01 limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,34658.40,162893.96,197552.36
1,37015.62,-85329.65,-48314.03
2,NaN,30199.73,30199.73
3,0.00,6599.16,6599.16
4,0.00,35745.17,35745.17
5,0.00,4251.32,4251.32
6,1718.50,12648.93,14367.43
7,2132.94,6689.14,8822.08
8,0.00,5310.61,5310.61
9,0.00,24290.45,24290.45


In [4]:
%%sql
CREATE OR REPLACE TABLE df_train AS
SELECT 
    *,
    (IFNULL(mpayroll, 0) - IFNULL(mcuentas_saldo, 0)) / NULLIF(ctrx_quarter, 0) AS intensidad_gasto
FROM df_train;



UsageError: Cell magic `%%sql` not found.


In [5]:
%%sql
select distinct
    Master_msaldototal
    , Visa_msaldototal
    , suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10


UsageError: Cell magic `%%sql` not found.


TAREA: Escriba una macro para hacer un ratio de dos variables que sea seguro, donde no solo hay campos con null, también esta el problema de la división por cero. Como es costumbre comparta su solución por este canal. Lea https://duckdb.org/docs/sql/functions/numeric.html para referencias de funciones que puede usar.

---

"Claro!" me dirá, mientras lee esto con un mate en la mano, "para cosas fáciles usar SQL alcanza, pero para algo más complicado como crear campos contra el data drifting es difícil".... elija su medicina:

In [21]:
%%sql
select
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad
from competencia_01
order by numero_de_cliente, cliente_antiguedad


,foto_mes,numero_de_cliente,cliente_antiguedad,cliente_antiguedad_2,cliente_antiguedad_3,cliente_antiguedad_4,cliente_antiguedad_5,cliente_antiguedad_6
0,202101,249221109,300,1,0.947564,0.947947,4,10
1,202102,249221109,301,2,0.947831,0.948213,4,10
2,202103,249221109,302,3,0.948291,0.948670,4,10
3,202104,249221109,303,4,0.948638,0.949016,4,10
4,202105,249221109,304,5,0.948950,0.949327,4,10
...,...,...,...,...,...,...,...,...
981941,202106,1598444941,1,1,0.000000,0.001553,1,1
981942,202106,1598471047,1,1,0.000000,0.001553,1,1
981943,202106,1598515897,1,1,0.000000,0.001553,1,1
981944,202106,1598517059,1,1,0.000000,0.001553,1,1


Qué paso? use las hermosas funciones analíticas de SQL. Al campo cliente_antiguedad (que no sufre de data drifting, solo esta para dar el ejemplo) para cada período (partition by foto_mes) la ordeno (order by cliente_antiguedad) y luego calculo las métricas de orden que pueden encontrar acá https://duckdb.org/docs/sql/window_functions.html#general-purpose-window-functions.

Seguiremos usando las funciones analíticas de SQL, esta vez para calcular features que utilizan valores del pasado.

Qué pasa si quiero agregar un feature que muestre el valor del periodo anterior?


In [22]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_01
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter
0,249270768,202101,52,NaN
1,249270768,202102,50,52.0
2,249270768,202103,51,50.0
3,249270768,202104,55,51.0
4,249270768,202105,51,55.0
5,249270768,202106,53,51.0
6,249280836,202101,28,NaN
7,249280836,202102,30,28.0
8,249280836,202103,28,30.0
9,249280836,202104,22,28.0


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [23]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter
0,249232117,202101,32,NaN,NaN,NaN
1,249232117,202102,35,32.0,3.0,NaN
2,249232117,202103,35,35.0,0.0,3.0
3,249232117,202104,34,35.0,-1.0,-1.0
4,249232117,202105,30,34.0,-4.0,-5.0
5,249232117,202106,29,30.0,-1.0,-5.0
6,249242331,202101,95,NaN,NaN,NaN
7,249242331,202102,92,95.0,-3.0,NaN
8,249242331,202103,90,92.0,-2.0,-5.0
9,249242331,202104,103,90.0,13.0,11.0


Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil con los últimos 3 meses anteriores? se puede hacer fácilmente


In [24]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 3 preceding and current row) as avg_3_ctrx_quarter
from competencia_01
order by numero_de_cliente, foto_mes desc
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,lag_2_ctrx_quarter,lag_3_ctrx_quarter,avg_3_ctrx_quarter
0,249221109,202106,199,199.0,188.0,174.0,190.000000
1,249221109,202105,199,188.0,174.0,161.0,180.500000
2,249221109,202104,188,174.0,161.0,166.0,172.250000
3,249221109,202103,174,161.0,166.0,NaN,167.000000
4,249221109,202102,161,166.0,NaN,NaN,163.500000
5,249221109,202101,166,NaN,NaN,NaN,166.000000
6,249221468,202106,191,182.0,182.0,170.0,181.250000
7,249221468,202105,182,182.0,170.0,154.0,172.000000
8,249221468,202104,182,170.0,154.0,145.0,162.750000
9,249221468,202103,170,154.0,145.0,NaN,156.333333


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis



In [25]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
  , max(ctrx_quarter) over ventana_3 as ctrx_quarter_max_3
  , min(ctrx_quarter) over ventana_3 as ctrx_quarter_min_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_media_3,ctrx_quarter_max_3,ctrx_quarter_min_3
0,249246268,202101,29,29.000000,29,29
1,249246268,202102,27,28.000000,29,27
2,249301400,202101,53,53.000000,53,53
3,249301400,202102,50,51.500000,53,50
4,249301400,202103,40,47.666667,53,40
5,249301400,202104,38,45.250000,53,38
6,249301400,202105,30,39.500000,50,30
7,249301400,202106,29,34.250000,40,29
8,249331084,202101,12,12.000000,12,12
9,249331084,202102,21,16.500000,21,12


Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [26]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_slope_3
0,249267856,202101,667,NaN
1,249267856,202102,648,-19.0
2,249267856,202103,698,15.5
3,249267856,202104,678,8.3
4,249267856,202105,653,-0.5
5,249267856,202106,573,-40.0
6,249445798,202101,144,NaN
7,249445798,202102,132,-12.0
8,249445798,202103,111,-16.5
9,249445798,202104,83,-20.4


... Alguno dirá "tenemos que escribir todo esto a mano? Son muchas variables!". Bueno no, use los conocimientos de programación para que la computadora trabaje para usted. Si tenemos una lista de campos


In [27]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad']


Podemos hacer un script muy sencillo que nos genere el texto que hay que poner en una query para generar esas variables


In [28]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as ctrx_{campo}_slope_3"
print(nuevos_features)



, regr_slope(active_quarter, cliente_antiguedad) over ventana_3 as ctrx_active_quarter_slope_3
, regr_slope(cliente_vip, cliente_antiguedad) over ventana_3 as ctrx_cliente_vip_slope_3
, regr_slope(internet, cliente_antiguedad) over ventana_3 as ctrx_internet_slope_3
, regr_slope(cliente_edad, cliente_antiguedad) over ventana_3 as ctrx_cliente_edad_slope_3
, regr_slope(cliente_antiguedad, cliente_antiguedad) over ventana_3 as ctrx_cliente_antiguedad_slope_3
, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_3 as ctrx_mrentabilidad_slope_3





Con la salida de esa celda, arme la query agregando las nuevas líneas y la ejecuta.

Lo que acabamos de hacer de manera muy simple es como "funcionan" sistemas como **dbt** que están tan de moda en el mundo de los datos.

La última reflexión, la creación de nuevas features es un proceso computacionalmente rápido pero intenso. Si ejecutó lo anterior pudo haber visto que en poco minutos tenía sus nuevas variables. Pero, también pudo haberle fallado por temas de recursos. Miles de variables necesitan los recursos adecuados. Use la nube, una máquina grande, al menos que sepa bien como optimizar las queries.


Y a no olvidarse guardar las nueva tabla

In [29]:
%%sql
COPY competencia_01 TO '{dataset_path}competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE);


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


: 